Let's say I want to play with a Matrix, ie edit cells, add or drop rows or columns.
First, I will import the Grid View Widget (as a matrix is grid-representable), and pick a matrix.

In [ ]:
from sage_combinat_widgets import *
from sage_widget_adapters.generic_grid_view_adapter import GridViewAdapter

In [ ]:
from sage.matrix.constructor import Matrix
m = Matrix(QQ, 3, 4, range(12))
m

Let's try the generic adapter.

In [ ]:
ma = GridViewAdapter()
GridViewWidget(m,ma)

To be able to display this matrix with a grid representation, we need to write an adapter.
First, we need to tell the widget which type are our matrix cells, and which would be a default cell content.

In [ ]:
class MyAdapter(GridViewAdapter):
    celltype = Integer
    cellzero = Integer(0)

In [ ]:
ma = MyAdapter()
GridViewWidget(m,ma)

We need to tell the widget which cells are to be displayed.

In [ ]:
class MyAdapter(GridViewAdapter):
    celltype = Integer
    cellzero = Integer(0)

    # See the documentation to check methods types and signatures
    @staticmethod
    def compute_cells(obj):
        """Compute cell positions."""
        from itertools import product
        return {(i,j):obj[i][j] for (i,j) in product(range(obj.nrows()), range(len(obj[0])))}

In [ ]:
ma = MyAdapter()
GridViewWidget(m,ma)

As our matrix widget should be editable, we need accessors for cell contents, and finally a method to get a new matrix value.

In [ ]:
class MyAdapter(GridViewAdapter):
    celltype = Integer
    cellzero = Integer(0)

    # See the documentation to check methods types and signatures
    @staticmethod
    def compute_cells(obj):
        """Compute cell positions."""
        from itertools import product
        return {(i,j):obj[i][j] for (i,j) in product(range(obj.nrows()), range(len(obj[0])))}

    @staticmethod
    def get_cell(obj, pos):
        """Return cell content"""
        if pos[0] >= obj.nrows() or pos[1] >= obj.ncols():
            raise ValueError("Entry '%s' does not exist!" % pos)
        return obj[pos[0]][pos[1]]

    @classmethod
    def set_cell(cls, obj, pos, val):
        """Set cell content to `val`"""
        if not val in obj.base_ring():
            raise TypeError("Value '%s' is not compatible!" % val)
        obj.set_block(pos[0], pos[1], matrix(obj.base_ring(), 1, 1, val))
        return obj
    
    @classmethod
    def from_cells(cls, cells={}):
        """Compute a new matrix from a cells dictionary `cells`"""
        nrows = max(pos[0]+1 for pos in cells)
        ncols = max(pos[1]+1 for pos in cells)
        return matrix([[cells[(i,j)] for j in range(ncols)] for i in range(nrows)])


In [ ]:
ma = MyAdapter()
w = GridViewWidget(m,ma)
w

In [ ]:
w.value

In [ ]:
from ipywidgets import HBox, Label
m_input = GridViewWidget(m)
@interact
def f(x = m_input):
    return HBox((GridViewWidget(x),Label('>>>>'), GridViewWidget(x*2)))